In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/202302 BIE Technical Interview PLD2.csv')

C:\Users\Holden\AppData\Local\Temp\ipykernel_21820\137737204.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/202302 BIE Technical Interview PLD2.csv')


In [3]:
df.head()

,shipmonth,shippingzipcode
0,2,77201ASS
1,2,77201
2,2,77201
3,2,77201
4,2,77201


In [180]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, VarcharType
import os
from pyspark.sql.functions import sum, count, countDistinct


spark = SparkSession.builder \
    .appName('ETL Pipeline') \
    .getOrCreate()
# Define schema
schema = StructType([
    StructField('shipmonth', IntegerType(), True),
    StructField('shippingzipcode', VarcharType(10), True)
])

file_path = 'data/202302 BIE Technical Interview PLD2.csv'

file_ext = os.path.splitext(file_path)[1].lower()

file_format = {
    '.csv': 'csv',
    '.json': 'json',
    '.parquet': 'parquet',
    '.xlsx': 'com.crealytics.spark.excel'
}.get(file_ext, None)

if not file_format:
    raise ValueError('Unsupported file format.')

# Read the file into a Spark dataframe
df = spark.read \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .format(file_format) \
    .load(file_path)

expected_columns = set([field.name for field in schema.fields])
actual_columns = set(df.columns)

assert schema.fields[0].dataType == IntegerType(), 'Column "shipmonth" contains an invalid datatype'

# Validate the loaded DataFrame against the expected schema

assert expected_columns.issubset(actual_columns), 'The file does not contain the expected columns.'


In [181]:
# transform
grouped_df = df.groupBy("shipmonth").agg(
    count("shippingzipcode").alias("total_shippings"),
    countDistinct("shippingzipcode").alias("total_zipcodes")
)

In [ ]:
sfOptions = {
    "sfURL": "<your_snowflake_account>.snowflakecomputing.com",
    "sfUser": "<your_snowflake_username>",
    "sfPassword": "<your_snowflake_password>",
    "sfDatabase": "<your_snowflake_database>",
    "sfSchema": "<your_snowflake_schema>",
    "sfWarehouse": "<your_snowflake_warehouse>",
    "sfRole": "<your_snowflake_role>"
}

In [182]:
grouped_df.show()

+---------+---------------+--------------+
|shipmonth|total_shippings|total_zipcodes|
+---------+---------------+--------------+
|        3|         728660|         15908|
|        2|         602902|         15370|
+---------+---------------+--------------+



In [183]:
import net.snowflake.spark.snowflake.Utils.SNOWFLAKE_SOURCE_NAME

ModuleNotFoundError: No module named 'net'

In [184]:
import snowflake.connector

In [205]:
PASSWORD = os.getenv('SNOWSQL_PWD')
WAREHOUSE = os.getenv('WAREHOUSE')
ACCOUNT = os.getenv('ACCOUNT')

In [200]:
os.environ["SNOWSQL_PWD"] = '8WMkq9bUX3!'

In [203]:
os.environ["WAREHOUSE"] = 'COMPUTE_WH'

In [204]:
os.environ["ACCOUNT"] = 'byb28753'

In [207]:
print(ACCOUNT)

byb28753


In [206]:
conn = snowflake.connector.connect(
    user='holdenms',
    password=PASSWORD,
    account=ACCOUNT,
    warehouse=WAREHOUSE,
    database='DEV_RAW',
    schema='PUBLIC'
    )

ForbiddenError: 250001 (08001): Failed to connect to DB. Verify the account name is correct: byb28753.snowflakecomputing.com:443. 000403: HTTP 403: Forbidden